## 优化模型参数
### 超参

是可以调整的参数，可以 控制 模型训练优化 的 过程

不同的超参可能会影响 模型训练 和 收敛速度

一般会定义以下用于训练的超参：
 - 训练轮次（epoch）：训练是遍历数据集的次数
 - 批次大小（batch size）：数据集进行分批读取训练，设定每个批次数据的大小
 - 学习率（learning rate）：学习率偏小 可能收敛过慢  学习率过大有可能会不收敛或者不可预测的结果
  

In [1]:
epoch = 5
batch_size = 64
learning_rate = 1e-3 

## 损失函数

损失函数用来评价模型的**预测值**与**真实值**偏离的程度

使用绝对误差损失函数L1Loss

还有很多Loss函数

In [2]:
import numpy as np
import mindspore.nn as nn
from mindspore import Tensor
from mindspore import dtype as mstype

loss = nn.L1Loss()
output_data = Tensor(np.array([[1,2,3],[2,3,4]]),dtype = mstype.float32)
target_data = Tensor(np.array([[0,2,5],[3,1,1]]),dtype = mstype.float32)
print(loss(output_data,target_data))

1.5


## 优化器
优化器用于 计算 和 更新 梯度

模型优化算法的选择直接关系到最终模型的的性能

效果不好，未必是特征和模型设计的问题，很有可能是优化器的问题

MindSpore所有优化器都封装在`Optimizer`对象中 

本次例子中我们使用SGD优化器

另有其他很多优化器在`mindspore.nn.optim`中

构建一个`Optimizer`对象，这个对象能够保持当前参数并给予计算得到的参数进行参数更新

他需要一个可以优化需要优化的参数（Variable对象）的迭代器 例如网络中所有可以训练的`parameter`

将`params`设置为`net.trainable_params()`即可

然后设置Optimizer的参数选项 学习率 权重衰减等

样例：
```python
from mindspore import nn
opyim=nn.SGD(params = net.trainable_params(),learning_rate=0.1,weight_decay = 0.0)
```


## 训练
一般有四个步骤
 - 定义神将网络
 - 构建数据集
 - 定义超参、损失函数、优化器
 - 输入训练轮次和数据集 进行训练
   

In [1]:
import numpy as np
import mindspore
import mindspore.nn as nn
from mindspore import Tensor,Model
from mindspore import dtype as mstype
import mindspore.dataset as ds
import os
DATA_DIR = "./datasets/MNIST_Data/train"

if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)
#采样器
sampler = ds.SequentialSampler(num_samples=5)

# dataset = ds.Cifar100Dataset(DATA_DIR,sampler=sampler)

dataset = ds.MnistDataset(DATA_DIR,sampler=sampler) #这个是MNIST数据集

class LeNet5(nn.Cell):
    def __init__(self,num_class=10,num_channel = 1):
    # 初始化网络
        super(LeNet5,self).__init__()
        # 定义所需要的运算
        self.conv1 = nn.Conv2d(num_channel,6,5,pad_mode='valid') # 卷积
        self.conv2 = nn.Conv2d(6,16,5,pad_mode='valid')
        self.fc1 = nn.Dense(16*5*5,120) # 全连接层
        self.fc2 = nn.Dense(120,84)
        self.fc3 = nn.Dense(84,num_class)
        self.max_pool2d = nn.MaxPool2d(kernel_size=2,stride=2)# 最大池化-降采样
        self.relu = nn.ReLU() # 激活函数
        self.flatten = nn.Flatten()# flatten 扁平的意思=> 将原来的高维数组换成只有 一行 的数组 列数是之前的各维度之积

    # 定义网络构建函数
    def construct(self,x):
        # 构建前向网络
        x = self.conv1(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x    


net = LeNet5()